In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("HeartAnalysis") \
    .config("spark.driver.memory", "2g") \
    .getOrCreate()
 
print("Spark version:", spark.version)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/20 19:37:55 WARN Utils: Your hostname, Ninas-macbook.local, resolves to a loopback address: 127.0.0.1; using 10.124.248.129 instead (on interface en0)
25/11/20 19:37:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/20 19:37:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/20 19:37:56 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/11/20 19:37:56 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


Spark version: 4.0.1


In [2]:
cvd_df = spark.read.csv("heart.csv", header=True, inferSchema=True)
cvd_df.show(5)

+---+---+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+
|Age|Sex|ChestPainType|RestingBP|Cholesterol|FastingBS|RestingECG|MaxHR|ExerciseAngina|Oldpeak|ST_Slope|HeartDisease|
+---+---+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+
| 40|  M|          ATA|      140|        289|        0|    Normal|  172|             N|    0.0|      Up|           0|
| 49|  F|          NAP|      160|        180|        0|    Normal|  156|             N|    1.0|    Flat|           1|
| 37|  M|          ATA|      130|        283|        0|        ST|   98|             N|    0.0|      Up|           0|
| 48|  F|          ASY|      138|        214|        0|    Normal|  108|             Y|    1.5|    Flat|           1|
| 54|  M|          NAP|      150|        195|        0|    Normal|  122|             N|    0.0|      Up|           0|
+---+---+-------------+---------+-----------+---------+-

In [3]:
from pyspark.sql.functions import col, when, count

null_counts = cvd_df.select([
    count(when(col(c).isNull(), c)).alias(c) for c in cvd_df.columns
])

null_counts.show()

+---+---+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+
|Age|Sex|ChestPainType|RestingBP|Cholesterol|FastingBS|RestingECG|MaxHR|ExerciseAngina|Oldpeak|ST_Slope|HeartDisease|
+---+---+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+
|  0|  0|            0|        0|          0|        0|         0|    0|             0|      0|       0|           0|
+---+---+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+



In [17]:

cvd_df = cvd_df.withColumn("HeartDisease", col("HeartDisease").cast("int"))


In [18]:
categorical_cols = [
    "Sex", "ChestPainType", "RestingECG",
    "ExerciseAngina", "ST_Slope"
]

numeric_cols = [
    "Age", "RestingBP", "Cholesterol",
    "FastingBS", "MaxHR", "Oldpeak"
]

label_col = "HeartDisease"

In [19]:
indexers = [
    StringIndexer(inputCol=c, outputCol=f"{c}_idx")
    for c in categorical_cols
]


In [20]:
encoder = OneHotEncoder(
    inputCols=[f"{c}_idx" for c in categorical_cols],
    outputCols=[f"{c}_vec" for c in categorical_cols]
)

In [21]:
assembler = VectorAssembler(
    inputCols=numeric_cols + [f"{c}_vec" for c in categorical_cols],
    outputCol="features"
)

In [22]:
lr = LogisticRegression(featuresCol="features", labelCol=label_col)

In [23]:
pipeline = Pipeline(stages=indexers + [encoder, assembler, lr])

In [24]:
model = pipeline.fit(train_df)

25/11/20 19:44:47 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/11/20 19:44:48 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
